In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle

from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, adjusted_rand_score, adjusted_mutual_info_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import SpectralClustering

from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans

from utils import smooth_time_series, compute_similarity
from config import get_arguments
from models import ClusterNetSmooth, ClusterNet, TAE
from load_data import get_loader

from train import training_function, pretrain_autoencoder, initalize_centroids, kl_loss_function
from sklearn.cluster import SpectralClustering

# Baseline clustering

## K-means clustering 

In this section, we wish to perform K-means clustering over the time-series. We smooth the time-series using a rolling average window. 
We first study the example of the MoteStrain dataset and then predict over all studied UCR datasets.

In [ ]:
reports_save_path = './AnalysisResults/'
if not os.path.exists(reports_save_path):
    os.makedirs(reports_save_path)
reports_save_path = './AnalysisResults/ExploratoryDataAnalysis/'
if not os.path.exists(reports_save_path):
    os.makedirs(reports_save_path)

In [2]:

def load_ucr(dataset, scale):
    """
    Load ucr dataset.
    Taken from https://github.com/FlorentF9/DeepTemporalClustering/blob/4f70d6b24722bd9f8331502d9cae00d35686a4d2/datasets.py#L18
    """
    X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset)
    X = np.concatenate((X_train, X_test))
    y = np.concatenate((y_train, y_test))
    if dataset == "HandMovementDirection":  # this one has special labels
        y = [yy[0] for yy in y]
    y = LabelEncoder().fit_transform(y)  # sometimes labels are strings or start from 1
    assert y.min() == 0  # assert labels are integers and start from 0
    # preprocess data (standardization)
    if scale:
        X = TimeSeriesScalerMeanVariance().fit_transform(X)
    return X, y


In [3]:
data = {
    "BeetleFly": 8,
    "BirdChicken": 8,
    "Computers": 10,
    "Earthquakes": 8,
    "MoteStrain": 4,
    "PhalangesOutlinesCorrect": 4,
    "ProximalPhalanxOutlineCorrect": 4,
    "ShapeletSim": 10,
    "ItalyPowerDemand": 4,
    "WormsTwoClass": 10,
}


In [4]:

class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        

In [6]:
args_dict = {
    "dataset_name": "MoteStrain",
    "smooth": True,
    "window_size": 10,
    "step_size": 1,
    "metric": "dtw",
}

args = Args(**args_dict)

In [7]:

ucr = UCR_UEA_datasets()
all_ucr_datasets = ucr.list_datasets()
X_scaled, y = load_ucr(args.dataset_name, all_ucr_datasets)


### Smoothing the dataset

In [8]:
if args.smooth:
    X_scaled = smooth_time_series(X_scaled, args.window_size, args.step_size)

### Predicting

In [9]:
model = TimeSeriesKMeans(n_clusters=2, metric=args.metric,
                         max_iter=10, random_state=0)
model.fit(X_scaled)

TimeSeriesKMeans(max_iter=10, metric='dtw', n_clusters=2, random_state=0)

In [10]:
res = model.predict(X_scaled)

/home/vayvi/.local/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 1272 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


In [11]:
roc_auc = max(
                roc_auc_score(y, res),
                roc_auc_score(y, 1 - res),
            )
print(roc_auc)

0.82989592011838


### Predicting over all datasets
Please note that as this is for multiple datasets over multiple metrics, the following section takes approximately 2 hours to run. 
We therefore set the variable evaluation_loop to False. If you would like to reproduce the results accross all datasets, set to True. 

In [1]:
evaluation_loop = False

In [ ]:
if evaluation_loop:
    rows = []
    for dataset in data.keys():
        for metric in ["dtw", "softdtw", "euclidean"]:
            print(dataset)
            args_dict = {
                "dataset_name": dataset,
                "smooth": True,
                "window_size": 10,
                "step_size": 1,
                "metric": metric,
            }

            args = Args(**args_dict)

            ucr = UCR_UEA_datasets()
            all_ucr_datasets = ucr.list_datasets()
            X_scaled, y = load_ucr(args.dataset_name, all_ucr_datasets)

            if args.smooth:
                X_scaled = smooth_time_series(X_scaled, args.window_size, args.step_size)

            model = TimeSeriesKMeans(n_clusters=2, metric=args.metric,
                                    max_iter=10, random_state=0)
            model.fit(X_scaled)
            res = model.predict(X_scaled)
            roc_auc = max(
                            roc_auc_score(y, res),
                            roc_auc_score(y, 1 - res),
                        )
            print(roc_auc)
            rows.append(list(args_dict.values())  + [roc_auc])
    df1 = pd.DataFrame(data=rows, columns = list(args_dict.keys())  + ["roc auc"])
    df1.to_csv(f"ROC_kmeans_clustering_{args_dict['window_size']}_{args_dict['step_size']}.csv")

BeetleFly


/home/vayvi/.local/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 40 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


0.5
BeetleFly


/home/vayvi/.local/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 40 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


0.625
BeetleFly


...

/home/vayvi/.local/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 258 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


0.5504279293146973
WormsTwoClass


/home/vayvi/.local/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 258 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


0.5714549596699711
WormsTwoClass
0.5366356751431561


/home/vayvi/.local/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 258 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


# Ablation Studies  
## Deep clustering without AutoEncoding:
Example for training the network without the autoencoder

We provide an example for training the deep clustering network over the MoteStrain datset. 

#### Model parameters

In [2]:
# general parameters
similarity="EUC"
window_size=10
step =4
dataset_name = "MoteStrain"
command = f"--similarity {similarity} --pool {1} --dataset_name {dataset_name} --window_size {window_size} --step {step} --smooth --no_autoencoder --heatmap"
parser = get_arguments()

args = parser.parse_args(command.split(" "))

# save paths
args.path_data = args.path_data.format(args.dataset_name + str(window_size) + str(step) + str(similarity) )
if not os.path.exists(args.path_data):
    os.makedirs(args.path_data)

path_weights = args.path_weights.format(args.dataset_name + str(window_size) + str(step) + str(similarity) )
if not os.path.exists(path_weights):
    os.makedirs(path_weights)

args.path_weights_ae = os.path.join(path_weights, "autoencoder_weight.pth")
args.path_weights_main = os.path.join(path_weights, "full_model_weigths.pth")

args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:

trainloader, X_scaled = get_loader(args)

for e in trainloader:
    for a in e:
        print(f"x shape {a.shape}")
        n_hidden = a.shape[-1]
        break
    break
args.n_hidden = n_hidden

x shape torch.Size([256, 1, 18])


In [4]:
if args.no_autoencoder:
    ### Clustering without autoencoder
    model = ClusterNetSmooth(args)
    loss1 = None

model = model.to(args.device)
initalize_centroids(X_scaled, args, model)

optimizer_clu = torch.optim.SGD(
    model.parameters(), lr=args.lr_cluster, momentum=args.momentum
)
max_roc_score = training_function(
    trainloader, X_scaled, model, optimizer_clu, loss1, args
)


print(
    f"maximum roc score for {args.dataset_name} with {args.similarity} without the AutoEncoder with smoothing {args.smooth} is {max_roc_score}"
)


Training full model ...


/home/vayvi/.local/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


For epoch 0 Loss is :  0.785 

For epoch 1 Loss is :  0.462 

For epoch 2 Loss is :  0.159 

For epoch 3 Loss is :  0.060 

For epoch 4 Loss is :  0.058 

For epoch 5 Loss is :  0.044 

For epoch 6 Loss is :  0.017 

For epoch 7 Loss is :  0.006 

For epoch 8 Loss is :  0.007 

For epoch 9 Loss is :  0.007 

For epoch 10 Loss is :  0.004 

For epoch 11 Loss is :  0.001 

For epoch 12 Loss is :  0.000 

For epoch 13 Loss is :  0.001 

For epoch 14 Loss is :  0.001 

For epoch 15 Loss is :  0.000 

For epoch 16 Loss is :  0.000 

For epoch 17 Loss is :  0.000 

For epoch 18 Loss is :  0.000 

For epoch 19 Loss is :  0.000 

For epoch 20 Loss is :  0.000 

For epoch 21 Loss is :  0.000 

For epoch 22 Loss is :  0.000 

For epoch 23 Loss is :  0.000 

For epoch 24 Loss is :  0.000 

For epoch 25 Loss is :  0.000 

For epoch 26 Loss is :  0.000 

For epoch 27 Loss is :  0.000 

For epoch 28 Loss is :  0.000 

For epoch 29 Loss is :  0.000 

For epoch 30 Loss is :  0.000 

For epoch 31 Loss 

In [5]:

state_dict = torch.load(args.path_weights_main)
model.load_state_dict(state_dict)

<All keys matched successfully>

### Testing Deep clustering without AutoEncoder on multiple datasets

In [ ]:
data = {
    "BeetleFly": 8,
    "BirdChicken": 8,
    "Computers": 10,
    "Earthquakes": 8,
    "MoteStrain": 4,
    "PhalangesOutlinesCorrect": 4,
    "ProximalPhalanxOutlineCorrect": 4,
    "ShapeletSim": 10,
    "ItalyPowerDemand": 4,
    "WormsTwoClass": 10,
}



In [ ]:
if evaluation_loop:
    rows=[]
    for dataset_name, pool in data.items():
        for similarity  in ["EUC", "COR", "CID"]:
            for window_size in [5, 10]:
                for step in [1, 4]:
                    for exp_number in range(5):
                        try:
                            command = f"--similarity {similarity} --pool {1} --dataset_name {dataset_name} --window_size {window_size} --step {step} --smooth --no_autoencoder --heatmap"
                            parser = get_arguments()
                            args = parser.parse_args(command.split(" "))
                            args.path_data = args.path_data.format(args.dataset_name + str(window_size) + str(step) + str(similarity) )
                            if not os.path.exists(args.path_data):
                                os.makedirs(args.path_data)

                            path_weights = args.path_weights.format(args.dataset_name + str(window_size) + str(step) + str(similarity) )
                            if not os.path.exists(path_weights):
                                os.makedirs(path_weights)

                            args.path_weights_ae = os.path.join(path_weights, "autoencoder_weight.pth")
                            args.path_weights_main = os.path.join(path_weights, "full_model_weigths.pth")

                            args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

                            trainloader, X_scaled = get_loader(args)
                            for e in trainloader:
                                for a in e:
                                    print(f"x shape {a.shape}")
                                    n_hidden = a.shape[-1]
                                    break
                                break
                            args.n_hidden = n_hidden
                            if args.no_autoencoder:
                                ### Clustering without autoencoder
                                model = ClusterNetSmooth(args)
                                loss1 = None
                    
                            model = model.to(args.device)
                            initalize_centroids(X_scaled, args, model)
                            # I moved initialize_centroids because otherwise Optimzer has no parameters 
                            optimizer_clu = torch.optim.SGD(
                                model.parameters(), lr=args.lr_cluster, momentum=args.momentum
                            )
                            max_roc_score = training_function(
                                trainloader, X_scaled, model, optimizer_clu, loss1, args
                            )

                            print(
                                f"maximum roc score for {args.dataset_name} with {args.similarity} (no autoencoder {args.no_autoencoder}) (smoothing {args.smooth}) is {max_roc_score}"
                            )
                            rows.append([args.dataset_name, str(window_size), str(step), str(similarity), max_roc_score, exp_number ])            
                        # # load the saved state dictionary
                        # state_dict = torch.load(args.path_weights_main)

                        # # load the state dictionary into the model
                        # model.load_state_dict(state_dict)
                        except ValueError:
                            continue
    df = pd.DataFrame(data=rows, columns = ["Dataset Name", "Window Size", "Step", "Similarity", "ROC", "Experiment number"])
    final_df = df.groupby(["Dataset Name", "Window Size", "Step", "Similarity"]).agg({"ROC": "mean"}).reset_index()
    final_df.to_csv("ROC_table_for_deep_clustering.csv")



x shape torch.Size([40, 1, 508])
torch.Size([2, 508])
Training full model ...


/home/vayvi/.local/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


For epoch 0 Loss is :  0.957 

For epoch 1 Loss is :  0.956 

For epoch 2 Loss is :  0.956 

For epoch 3 Loss is :  0.955 

For epoch 4 Loss is :  0.955 

For epoch 5 Loss is :  0.954 

For epoch 6 Loss is :  0.953 

For epoch 7 Loss is :  0.951 

For epoch 8 Loss is :  0.950 

For epoch 9 Loss is :  0.949 

For epoch 10 Loss is :  0.947 

For epoch 11 Loss is :  0.946 

For epoch 12 Loss is :  0.944 

For epoch 13 Loss is :  0.942 

For epoch 14 Loss is :  0.941 

For epoch 15 Loss is :  0.939 

For epoch 16 Loss is :  0.937 

For epoch 17 Loss is :  0.935 

For epoch 18 Loss is :  0.933 

For epoch 19 Loss is :  0.931 

For epoch 20 Loss is :  0.929 

For epoch 21 Loss is :  0.927 

For epoch 22 Loss is :  0.925 

For epoch 23 Loss is :  0.923 

For epoch 24 Loss is :  0.921 

For epoch 25 Loss is :  0.919 

For epoch 26 Loss is :  0.917 

For epoch 27 Loss is :  0.915 

For epoch 28 Loss is :  0.913 

For epoch 29 Loss is :  0.910 

For epoch 30 Loss is :  0.908 

For epoch 31 Loss 

/home/vayvi/.local/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


For epoch 0 Loss is :  0.957 

For epoch 1 Loss is :  0.957 

For epoch 2 Loss is :  0.957 

For epoch 3 Loss is :  0.956 

For epoch 4 Loss is :  0.955 

For epoch 5 Loss is :  0.954 

For epoch 6 Loss is :  0.953 

For epoch 7 Loss is :  0.952 

For epoch 8 Loss is :  0.951 

For epoch 9 Loss is :  0.949 

For epoch 10 Loss is :  0.948 

For epoch 11 Loss is :  0.946 

For epoch 12 Loss is :  0.945 

For epoch 13 Loss is :  0.943 

For epoch 14 Loss is :  0.941 

For epoch 15 Loss is :  0.939 

For epoch 16 Loss is :  0.938 

For epoch 17 Loss is :  0.936 

For epoch 18 Loss is :  0.934 

For epoch 19 Loss is :  0.932 

For epoch 20 Loss is :  0.930 

For epoch 21 Loss is :  0.928 

For epoch 22 Loss is :  0.926 

For epoch 23 Loss is :  0.924 

For epoch 24 Loss is :  0.922 

For epoch 25 Loss is :  0.919 

For epoch 26 Loss is :  0.917 

For epoch 27 Loss is :  0.915 

For epoch 28 Loss is :  0.913 

For epoch 29 Loss is :  0.911 

For epoch 30 Loss is :  0.909 

For epoch 31 Loss 

...

/home/vayvi/.local/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


For epoch 0 Loss is :  0.014 

For epoch 1 Loss is :  0.009 

For epoch 2 Loss is :  0.010 

For epoch 3 Loss is :  0.013 

For epoch 4 Loss is :  0.010 

For epoch 5 Loss is :  0.008 

For epoch 6 Loss is :  0.010 

For epoch 7 Loss is :  0.011 

For epoch 8 Loss is :  0.008 

For epoch 9 Loss is :  0.008 

For epoch 10 Loss is :  0.010 

For epoch 11 Loss is :  0.009 

For epoch 12 Loss is :  0.007 

For epoch 13 Loss is :  0.008 

For epoch 14 Loss is :  0.008 

For epoch 15 Loss is :  0.007 

For epoch 16 Loss is :  0.007 

For epoch 17 Loss is :  0.008 

For epoch 18 Loss is :  0.007 

For epoch 19 Loss is :  0.007 

For epoch 20 Loss is :  0.007 

For epoch 21 Loss is :  0.007 

For epoch 22 Loss is :  0.006 

For epoch 23 Loss is :  0.006 

For epoch 24 Loss is :  0.006 

For epoch 25 Loss is :  0.006 

For epoch 26 Loss is :  0.006 

For epoch 27 Loss is :  0.006 

For epoch 28 Loss is :  0.006 

For epoch 29 Loss is :  0.005 

For epoch 30 Loss is :  0.005 

For epoch 31 Loss 

/home/vayvi/.local/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


For epoch 0 Loss is :  0.014 

For epoch 1 Loss is :  0.009 

For epoch 2 Loss is :  0.010 

For epoch 3 Loss is :  0.013 

For epoch 4 Loss is :  0.010 

For epoch 5 Loss is :  0.008 

For epoch 6 Loss is :  0.011 

For epoch 7 Loss is :  0.011 

For epoch 8 Loss is :  0.008 

For epoch 9 Loss is :  0.009 

For epoch 10 Loss is :  0.010 

For epoch 11 Loss is :  0.009 

For epoch 12 Loss is :  0.007 

For epoch 13 Loss is :  0.009 

For epoch 14 Loss is :  0.009 

For epoch 15 Loss is :  0.007 

For epoch 16 Loss is :  0.007 

For epoch 17 Loss is :  0.008 

For epoch 18 Loss is :  0.007 

For epoch 19 Loss is :  0.007 

For epoch 20 Loss is :  0.007 

For epoch 21 Loss is :  0.007 

For epoch 22 Loss is :  0.006 

For epoch 23 Loss is :  0.006 

For epoch 24 Loss is :  0.007 

For epoch 25 Loss is :  0.006 

For epoch 26 Loss is :  0.006 

For epoch 27 Loss is :  0.006 

For epoch 28 Loss is :  0.006 

For epoch 29 Loss is :  0.005 

For epoch 30 Loss is :  0.005 

For epoch 31 Loss 

NameError: name 'exit' is not defined

In [ ]:
final_df

,Dataset Name,Similarity,"ROC (5,1)","ROC (5, 4)","ROC (10,1)","ROC (10, 4)"
0,BeetleFly,CID,0.800000,0.800000,0.775000,0.780000
1,BeetleFly,COR,0.650000,0.650000,0.550000,0.575000
2,BeetleFly,EUC,0.625000,0.625000,0.575000,0.575000
3,BirdChicken,CID,0.675000,0.695000,0.675000,0.700000
4,BirdChicken,COR,0.575000,0.575000,0.575000,0.575000
5,BirdChicken,EUC,0.575000,0.575000,0.575000,0.575000
6,Computers,EUC,0.522000,0.506000,0.518000,0.528400
7,Earthquakes,CID,0.521073,0.542999,0.520456,0.544203
8,Earthquakes,COR,0.539855,0.539811,0.567622,0.586641
9,Earthquakes,EUC,0.509394,0.501359,0.505376,0.509394


In [ ]:
experiment_save_path = "./AnalysisResults/AblationStudy/"

## Temporal auto encoder (TAE) with spectral clustering

### Helper Functions for evaluation : 

In [ ]:
def get_metrics_from_preds(all_gt, all_preds):
    metrics = {}
    metrics['two_class_accuracy'] = max(
                roc_auc_score(all_gt, all_preds),
                roc_auc_score(all_gt, 1 - all_preds),
            )
    metrics['adjusted_rand_score'] = adjusted_rand_score(all_gt, all_preds)
    metrics['adjusted_mi_score'] = adjusted_mutual_info_score(all_gt, all_preds)
    return metrics

def eval_model(trainloader, model):
    with torch.no_grad():
        model.eval()
        eval_loss = 0
        all_preds, all_gt = [], []
        for batch_idx, (inputs, labels) in enumerate(trainloader):
            inputs = inputs.type(torch.FloatTensor).to(args.device)
            all_gt.append(labels.cpu().detach())
            z, x_reconstr, Q, P = model(inputs)
            loss_mse = loss1(inputs.squeeze(), x_reconstr)
            loss_KL = kl_loss_function(P, Q)
            total_loss = loss_mse + loss_KL
            preds = torch.max(Q, dim=1)[1]
            all_preds.append(preds.cpu().detach())
            eval_loss += total_loss.item()
            
    all_gt = torch.cat(all_gt, dim=0).numpy()
    all_preds = torch.cat(all_preds, dim=0).numpy()
    metrics = get_metrics_from_preds(all_gt, all_preds)
    metrics['totla_loss'] = eval_loss / (batch_idx + 1)
    return metrics
    
def test_classical_clustering(trainloader, model, second_clustering):
    with torch.no_grad():
        model.eval()
        all_zs, all_gt = [], []
        for batch_idx, (inputs, labels) in enumerate(trainloader):
            inputs = inputs.type(torch.FloatTensor).to(args.device)
            all_gt.append(labels.cpu().detach())
            z, x_reconstr, Q, P = model(inputs)
            all_zs.append(z.cpu().detach())
    all_zd = torch.cat(all_zs, dim=0)
    distance = compute_similarity(all_zd, all_zd, second_clustering).detach().numpy()     
    affinity = np.nan_to_num(np.exp(-(distance-distance.flatten().mean())/distance.flatten().std()))
    all_preds = torch.Tensor(SpectralClustering(n_clusters=args.n_clusters, affinity="precomputed").fit_predict(affinity))
    all_gt = torch.cat(all_gt, dim=0).numpy()
    metrics = get_metrics_from_preds(all_gt, all_preds)
    return metrics

### Evaluation Loop

In [ ]:
N_iters = 5
res_train_all_then_classical_clustering = {}
for dataset_name, pool in tqdm(data.items()):
    command = (
            f"--similarity EUC --pool {pool} --dataset_name {dataset_name}"
        )
    res_train_all_then_classical_clustering[(dataset_name, pool)] = {}
    parser = get_arguments()
    args = parser.parse_args(command.split(" "))
    args.path_data = args.path_data.format(args.dataset_name)

    for similarity in tqdm(['EUC']):
        command = (
            f"--similarity {similarity} --epochs_ae 100 --max_epochs 1 --pool {pool} --dataset_name {dataset_name}"
        )
        parser = get_arguments()
        print(command.split(" "))
        args = parser.parse_args(command.split(" "))
        args.path_data = args.path_data.format(args.dataset_name)

        if not os.path.exists(args.path_data):
            os.makedirs(args.path_data)

        path_weights = args.path_weights.format(args.dataset_name)
        if not os.path.exists(path_weights):
            os.makedirs(path_weights)

        args.path_weights_ae = os.path.join(path_weights, "autoencoder_weight.pth")
        args.path_weights_main = os.path.join(path_weights, "full_model_weigths.pth")

        args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        trainloader, X_scaled = get_loader(args)
        for second_clustering in tqdm(['EUC', 'COR', 'CID']):
            res_train_all_then_classical_clustering[(dataset_name, pool)][(similarity, second_clustering)] = []
            for _ in tqdm(range(N_iters), desc=f"similarity {similarity} clustering {second_clustering}") : 
                pretrain_autoencoder(trainloader, args, verbose=False)
                model = ClusterNet(args)
                model = model.to(args.device)
                loss1 = nn.MSELoss()
                optimizer_clu = torch.optim.SGD(
                    model.parameters(), lr=args.lr_cluster, momentum=args.momentum
                )

                max_roc_score = training_function(trainloader, X_scaled, model, optimizer_clu, loss1, args, verbose=False)

                print(
                    f"maximum roc score for {args.dataset_name} with {args.similarity} is {max_roc_score}"
                )
                res_1 = eval_model(trainloader, model)
                res_2 = test_classical_clustering(trainloader, model, second_clustering)
                res_train_all_then_classical_clustering[(dataset_name, pool)][(similarity, second_clustering)].append((res_1, res_2))
                with open(experiment_save_path + 'ablation_tae_training_with_spectral.pickle', 'wb+') as f:
                    pickle.dump(res_train_all_then_classical_clustering, f)
                    f.close()

In [ ]:
data = {
    "BeetleFly": 8,
    "BirdChicken": 8,
    "Computers": 10,
    "Earthquakes": 8,
    "MoteStrain": 4,
    "PhalangesOutlinesCorrect": 4,
    "ProximalPhalanxOutlineCorrect": 4,
    "ShapeletSim": 10,
    "ItalyPowerDemand": 4,
    "WormsTwoClass": 10,
}

In [ ]:
experiment_save_path = "./AnalysisResults/AblationStudy/"

## DTC with spectral clustering

### Helper Functions for evaluation : 

In [ ]:
def get_metrics_from_preds(all_gt, all_preds):
    metrics = {}
    metrics['two_class_accuracy'] = max(
                roc_auc_score(all_gt, all_preds),
                roc_auc_score(all_gt, 1 - all_preds),
            )
    metrics['adjusted_rand_score'] = adjusted_rand_score(all_gt, all_preds)
    metrics['adjusted_mi_score'] = adjusted_mutual_info_score(all_gt, all_preds)
    return metrics

def eval_model(trainloader, model):
    with torch.no_grad():
        model.eval()
        eval_loss = 0
        all_preds, all_gt = [], []
        for batch_idx, (inputs, labels) in enumerate(trainloader):
            inputs = inputs.type(torch.FloatTensor).to(args.device)
            all_gt.append(labels.cpu().detach())
            z, x_reconstr, Q, P = model(inputs)
            loss_mse = loss1(inputs.squeeze(), x_reconstr)
            loss_KL = kl_loss_function(P, Q)
            total_loss = loss_mse + loss_KL
            preds = torch.max(Q, dim=1)[1]
            all_preds.append(preds.cpu().detach())
            eval_loss += total_loss.item()
            
    all_gt = torch.cat(all_gt, dim=0).numpy()
    all_preds = torch.cat(all_preds, dim=0).numpy()
    metrics = get_metrics_from_preds(all_gt, all_preds)
    metrics['totla_loss'] = eval_loss / (batch_idx + 1)
    return metrics
    
def test_classical_clustering(trainloader, model, second_clustering):
    with torch.no_grad():
        model.eval()
        all_zs, all_gt = [], []
        for batch_idx, (inputs, labels) in enumerate(trainloader):
            inputs = inputs.type(torch.FloatTensor).to(args.device)
            all_gt.append(labels.cpu().detach())
            z, x_reconstr, Q, P = model(inputs)
            all_zs.append(z.cpu().detach())
    all_zd = torch.cat(all_zs, dim=0)
    distance = compute_similarity(all_zd, all_zd, second_clustering).detach().numpy()     
    affinity = np.nan_to_num(np.exp(-(distance-distance.flatten().mean())/distance.flatten().std()))
    all_preds = torch.Tensor(SpectralClustering(n_clusters=args.n_clusters, affinity="precomputed").fit_predict(affinity))
    all_gt = torch.cat(all_gt, dim=0).numpy()
    metrics = get_metrics_from_preds(all_gt, all_preds)
    return metrics

### Evaluation Loop

In [ ]:
N_iters = 5
res_train_all_then_classical_clustering = {}
for dataset_name, pool in tqdm(data.items()):
    command = (
            f"--similarity EUC --pool {pool} --dataset_name {dataset_name}"
        )
    res_train_all_then_classical_clustering[(dataset_name, pool)] = {}
    parser = get_arguments()
    args = parser.parse_args(command.split(" "))
    args.path_data = args.path_data.format(args.dataset_name)

    for similarity in tqdm(['EUC', 'COR', 'CID']):
        command = (
            f"--similarity {similarity} --pool {pool} --dataset_name {dataset_name}"
        )
        parser = get_arguments()
        print(command.split(" "))
        args = parser.parse_args(command.split(" "))
        args.path_data = args.path_data.format(args.dataset_name)

        if not os.path.exists(args.path_data):
            os.makedirs(args.path_data)

        path_weights = args.path_weights.format(args.dataset_name)
        if not os.path.exists(path_weights):
            os.makedirs(path_weights)

        args.path_weights_ae = os.path.join(path_weights, "autoencoder_weight.pth")
        args.path_weights_main = os.path.join(path_weights, "full_model_weigths.pth")

        args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        trainloader, X_scaled = get_loader(args)
        for second_clustering in tqdm(['EUC', 'COR', 'CID']):
            res_train_all_then_classical_clustering[(dataset_name, pool)][(similarity, second_clustering)] = []
            for _ in tqdm(range(N_iters), desc=f"similarity {similarity} clustering {second_clustering}") : 
                pretrain_autoencoder(trainloader, args, verbose=False)
                model = ClusterNet(args)
                model = model.to(args.device)
                loss1 = nn.MSELoss()
                optimizer_clu = torch.optim.SGD(
                    model.parameters(), lr=args.lr_cluster, momentum=args.momentum
                )

                max_roc_score = training_function(trainloader, X_scaled, model, optimizer_clu, loss1, args, verbose=False)

                print(
                    f"maximum roc score for {args.dataset_name} with {args.similarity} is {max_roc_score}"
                )
                
                res_1 = eval_model(trainloader, model)
                res_2 = test_classical_clustering(trainloader, model, second_clustering)
                res_train_all_then_classical_clustering[(dataset_name, pool)][(similarity, second_clustering)].append((res_1, res_2))
                
                with open(experiment_save_path + 'ablation_full_training_with_spectral.pickle', 'wb+') as f:
                    pickle.dump(res_train_all_then_classical_clustering, f)
                    f.close()